## Lab | Customer Analysis Round 6

For this lab, we still keep using the marketing_customer_analysis.csv file that you can find in the files_for_lab folder.

Get the data

We are using the marketing_customer_analysis.csv file.

Dealing with the data

Already done in the round 2.

Explore the data

Done in the round 3.

Processing Data

(Further processing...)

X-y split. (done)
Normalize (numerical). (done)
One Hot/Label Encoding (categorical).
Concat DataFrames
Linear Regression

Train-test split.
Apply linear regression.
Model Validation

Description:
R2.
MSE.
RMSE.
MAE.

In [26]:
import pandas as pd
import numpy as np
import warnings
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [29]:
df = pd.read_excel('/Users/patrickhutsch/IH-Labs/lab-customer-analysis-round-2/files_for_lab/excel_files/marketing_customer_analysis.xlsx')

In [30]:
data = df.copy()
data.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [31]:
cols = []
for i in range(len(data.columns)):
    cols.append(data.columns[i].lower().replace(' ', '_'))
data.columns = cols

print(data.columns)

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')


In [32]:
# data = df.copy()
# data.head()

In [33]:
data.shape
data

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,2011-01-06,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2011-02-13,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A


In [34]:
#X-y split
X = data.drop(['total_claim_amount'], axis=1)
y = data['total_claim_amount']

In [36]:
def boxcox_transform(X):
    numeric_cols = X.select_dtypes(np.number).columns #numerical variables
    _ci = {column: None for column in numeric_cols}
    for column in numeric_cols:
        # since i know any columns should take negative numbers, to avoid -inf in df
        X[column] = np.where(X[column]<=0, np.NAN, X[column]) 
        X[column] = X[column].fillna(X[column].mean())
        transformed_data, ci = stats.boxcox(X[column])
        X[column] = transformed_data
        _ci[column] = [ci] 
    return X, _ci

In [37]:
data, _ci = boxcox_transform(X)
X

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,vehicle_type
0,621.011982,DK49336,Arizona,2.728415,No,Basic,College,2011-02-18,Employed,M,...,22.277861,0.647766,1.481918,Corporate Auto,Corporate L3,Offer3,Agent,Four-Door Car,Medsize,NaN
1,0.000000,KX64629,California,2.681977,No,Basic,College,2011-01-18,Unemployed,F,...,13.011677,0.647766,0.000000,Personal Auto,Personal L3,Offer4,Call Center,Four-Door Car,Medsize,NaN
2,0.894663,LZ68649,Washington,2.777786,No,Basic,Bachelor,2011-02-10,Employed,M,...,14.946662,0.647766,0.608253,Personal Auto,Personal L3,Offer3,Call Center,SUV,Medsize,A
3,1.661524,XL78013,Oregon,2.791099,Yes,Extended,College,2011-01-11,Employed,M,...,1.670780,0.647766,0.608253,Corporate Auto,Corporate L3,Offer2,Branch,Four-Door Car,Medsize,A
4,2.355634,QA50777,Oregon,2.758195,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,14.946662,0.647766,1.368807,Personal Auto,Personal L2,Offer1,Branch,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,1014.667292,FE99816,Nevada,2.779210,No,Premium,Bachelor,2011-01-19,Unemployed,F,...,18.221035,0.647766,1.368807,Personal Auto,Personal L1,Offer3,Web,Luxury Car,Medsize,A
10906,1014.733214,KX53892,Oregon,2.733056,No,Basic,College,2011-01-06,Employed,F,...,27.294671,0.647766,1.293804,Personal Auto,Personal L3,Offer2,Branch,Four-Door Car,Medsize,A
10907,1014.799134,TL39050,Arizona,2.793163,No,Extended,Bachelor,2011-02-06,Employed,F,...,25.766946,0.647766,0.608253,Corporate Auto,Corporate L3,Offer1,Web,Luxury SUV,Medsize,NaN
10908,1014.865052,WA60547,California,2.769587,No,Premium,College,2011-02-13,Employed,F,...,13.406564,1.510315,1.293804,Personal Auto,Personal L1,Offer1,Branch,SUV,Medsize,A


In [38]:
numerical = X.select_dtypes(np.number)
numerical

,unnamed:_0,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,621.011982,2.728415,1241.652611,0.682723,3.709788,22.277861,0.647766,1.481918
1,0.000000,2.681977,1279.349323,0.682838,1.559649,13.011677,0.647766,0.000000
2,0.894663,2.777786,769.901284,0.683590,12.291841,14.946662,0.647766,0.608253
3,1.661524,2.791099,1258.312585,0.683553,5.000611,1.670780,0.647766,0.608253
4,2.355634,2.758195,802.453581,0.683758,7.083446,14.946662,0.647766,1.368807
...,...,...,...,...,...,...,...,...
10905,1014.667292,2.779210,1279.349323,0.684198,7.083446,18.221035,0.647766,1.368807
10906,1014.733214,2.733056,1441.066852,0.682873,3.709788,27.294671,0.647766,1.293804
10907,1014.799134,2.793163,1106.354143,0.684114,5.395331,25.766946,0.647766,0.608253
10908,1014.865052,2.769587,1484.972511,0.683989,7.083446,13.406564,1.510315,1.293804


In [39]:
categorical = X.select_dtypes(include = np.object)
# categorical_data = pd.get_dummies(categorical, drop_first=True) # if you need to save the encoding info, this won't do
# categorical_data
categorical

,customer,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size,vehicle_type
0,DK49336,Arizona,No,Basic,College,Employed,M,Suburban,Married,Corporate Auto,Corporate L3,Offer3,Agent,Four-Door Car,Medsize,NaN
1,KX64629,California,No,Basic,College,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer4,Call Center,Four-Door Car,Medsize,NaN
2,LZ68649,Washington,No,Basic,Bachelor,Employed,M,Suburban,Single,Personal Auto,Personal L3,Offer3,Call Center,SUV,Medsize,A
3,XL78013,Oregon,Yes,Extended,College,Employed,M,Suburban,Single,Corporate Auto,Corporate L3,Offer2,Branch,Four-Door Car,Medsize,A
4,QA50777,Oregon,No,Premium,Bachelor,Medical Leave,F,Suburban,Married,Personal Auto,Personal L2,Offer1,Branch,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,FE99816,Nevada,No,Premium,Bachelor,Unemployed,F,Suburban,Married,Personal Auto,Personal L1,Offer3,Web,Luxury Car,Medsize,A
10906,KX53892,Oregon,No,Basic,College,Employed,F,Urban,Married,Personal Auto,Personal L3,Offer2,Branch,Four-Door Car,Medsize,A
10907,TL39050,Arizona,No,Extended,Bachelor,Employed,F,Rural,Married,Corporate Auto,Corporate L3,Offer1,Web,Luxury SUV,Medsize,NaN
10908,WA60547,California,No,Premium,College,Employed,F,Urban,Divorced,Personal Auto,Personal L1,Offer1,Branch,SUV,Medsize,A


In [40]:
encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(categorical)

OneHotEncoder(drop='first')

In [41]:
encoded = encoder.transform(categorical).toarray()
encoded.shape # 

(10910, 9181)

In [42]:
X_1 = np.concatenate([numerical, encoded], axis=1)
X_1.shape

(10910, 9189)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size=0.3, random_state=42)

In [44]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [45]:
predictions = model.predict(X_test)
predictions.shape

(3273,)

In [46]:
rmse = mean_squared_error(y_test, predictions, squared=False) # or mse with squared=True
mse = mean_squared_error(y_test, predictions, squared=True)
mae = mean_absolute_error(y_test, predictions)
print("R2_score:", round(r2_score(y_test, predictions),2)) # or r2_score(Y, predictions)
print("RMSE:", rmse)
print("MAE:", mae)
print("MSE:", mse)

R2_score: 0.46
RMSE: 211.33601738162378
MAE: 138.53870317796097
MSE: 44662.912242725986
